In [1]:
import os
import cv2
import numpy as np
import pytesseract
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageEnhance
import shutil

Funzione process_image: 

Converte l'immagine in scala di grigi.
Applica la sogliatura adattiva per ottenere un'immagine binarizzata.
Rimuove il rumore dall'immagine binarizzata.
Applica operazioni morfologiche per migliorare la forma dei caratteri.
Estrae i caratteri rilevati dall'immagine e li salva come immagini separate nella cartella di output. Disegna anche dei rettangoli intorno ai caratteri nell'immagine originale per mostrare i caratteri rilevati.
Salva l'immagine originale con i rettangoli e l'immagine binarizzata con i caratteri rilevati.

In [2]:
import os
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox

def process_image(image_path, output_path):
    if not os.path.exists(image_path):
        print(f"Errore: il file {image_path} non esiste.")
        return

    im = cv2.imread(image_path)

    if im is None:
        print(f"Errore durante la lettura dell'immagine: {image_path}")
        return

    base_name = os.path.splitext(os.path.basename(image_path))[0]

    # Converti in scala di grigi
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding
    bw = cv2.adaptiveThreshold(im_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 8)

    # Denoise image
    bw = cv2.fastNlMeansDenoising(bw, None, 30, 7, 21)

    # Morphological operations to separate characters better
    kernel_dilate = np.ones((2, 2), np.uint8)
    kernel_erode = np.ones((1, 1), np.uint8)
    bw = cv2.erode(bw, kernel_erode, iterations=1)
    bw = cv2.dilate(bw, kernel_dilate, iterations=1)

    # Find contours to detect characters
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sorting contours from left to right
    contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

    # Save the image without rectangles
    cv2.imwrite(os.path.join(output_path, f'{base_name}_out_without_rectangles.png'), im_gray)

    # Draw rectangles on the original image
    im_with_rectangles = im.copy()
    for i, ctr in enumerate(contours):
        x, y, w, h = cv2.boundingRect(ctr)
        char_img = im_gray[y:y+h, x:x+w]
        cv2.imwrite(os.path.join(output_path, f'{base_name}_char_{i}.jpg'), char_img)
        cv2.rectangle(im_with_rectangles, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Save the images with and without rectangles
    cv2.imwrite(os.path.join(output_path, f'{base_name}_out.png'), im_with_rectangles)
    cv2.imwrite(os.path.join(output_path, f'{base_name}_out_bw.png'), bw)

def select_input_directory():
    dir_path = filedialog.askdirectory(title="Seleziona la directory di input")
    if dir_path:
        input_dir.set(dir_path)

def select_output_directory():
    dir_path = filedialog.askdirectory(title="Seleziona la directory di output")
    if dir_path:
        output_dir.set(dir_path)

def process_all_images():
    input_path = input_dir.get()
    output_path = output_dir.get()
    if not input_path or not output_path:
        messagebox.showwarning("Attenzione", "Per favore seleziona sia la directory di input che quella di output")
        return

    supported_formats = (".tif", ".jpg", ".jpeg", ".png")
    for filename in os.listdir(input_path):
        if filename.lower().endswith(supported_formats): 
            full_input_path = os.path.join(input_path, filename)
            full_output_path = os.path.join(output_path, filename.split('.')[0])
            os.makedirs(full_output_path, exist_ok=True)
            process_image(full_input_path, full_output_path) 

    messagebox.showinfo("Completato", "Elaborazione completata!")

app = tk.Tk()
app.title("Image Processing Batch")

input_dir = tk.StringVar()
output_dir = tk.StringVar()

frame = tk.Frame(app)
frame.pack(padx=10, pady=10)

input_label = tk.Label(frame, text="Directory di input:")
input_label.grid(row=0, column=0, sticky="e")
input_entry = tk.Entry(frame, textvariable=input_dir, width=50)
input_entry.grid(row=0, column=1, padx=5)
input_button = tk.Button(frame, text="Sfoglia...", command=select_input_directory)
input_button.grid(row=0, column=2)

output_label = tk.Label(frame, text="Directory di output:")
output_label.grid(row=1, column=0, sticky="e")
output_entry = tk.Entry(frame, textvariable=output_dir, width=50)
output_entry.grid(row=1, column=1, padx=5)
output_button = tk.Button(frame, text="Sfoglia...", command=select_output_directory)
output_button.grid(row=1, column=2)

process_button = tk.Button(app, text="Processa tutte le immagini", command=process_all_images)
process_button.pack(pady=10)

app.mainloop()



In [11]:
drawing = False
ix, iy = -1, -1
fx, fy = -1, -1

def draw_rectangle(event, x, y, flags, param):
    global ix, iy, fx, fy, drawing, im_contours

    output_dir, characters, image_name = param

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            fx, fy = x, y

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        fx, fy = x, y

        # Controllo per assicurarsi che i punti del rettangolo siano all'interno dei limiti dell'immagine
        if ix < 0:
            ix = 0
        if iy < 0:
            iy = 0
        if fx >= im.shape[1]:
            fx = im.shape[1] - 1
        if fy >= im.shape[0]:
            fy = im.shape[0] - 1
        
        if ix < fx and iy < fy:
            cv2.rectangle(im_contours, (ix, iy), (fx, fy), (0, 0, 255), 2)
            roi = im[iy:fy, ix:fx]
            output_path = os.path.join(output_dir, f'{image_name}_character_{len(characters) + 1}.png')
            cv2.imwrite(output_path, roi)
            characters.append(output_path)

def detect_characters(image_path, output_dir=None):
    global im, im_contours

    if output_dir is None:
        output_dir = 'extracted_characters'
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    im = cv2.imread(image_path)
    if im is None:
        print(f"Errore durante la lettura dell'immagine: {image_path}")
        return

    h, w = im.shape[:2]
    im_contours = im.copy()
    im = cv2.cvtColor(im_contours, cv2.COLOR_BGR2GRAY)
    window_name = 'Contours with Bounding Boxes'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name,200, 100)

    image_name = os.path.splitext(os.path.basename(image_path))[0]
    output_dir = output_dir_entry.get()
    characters = []

    cv2.setMouseCallback(window_name, draw_rectangle, param=(output_dir, characters, image_name))

    while True:
        cv2.imshow(window_name, im)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('0'):
            break

    cv2.destroyAllWindows()

def select_image_file():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.tif *.bmp")])
    if file_path:
        image_path_entry.delete(0, tk.END)
        image_path_entry.insert(0, file_path)

def select_output_directory():
    dir_path = filedialog.askdirectory()
    if dir_path:
        output_dir_entry.delete(0, tk.END)
        output_dir_entry.insert(0, dir_path)

def start_extraction():
    image_path = image_path_entry.get()
    output_dir = output_dir_entry.get()
    
    if not image_path or not output_dir:
        messagebox.showerror("Errore", "Seleziona un'immagine e una directory di output.")
        return
    
    detect_characters(image_path, output_dir)
    messagebox.showinfo("Successo", "Estrazione dei caratteri completata.")

# Configurazione dell'interfaccia grafica
root = tk.Tk()
root.title("Estrattore di Caratteri")
root.geometry("600x200")

tk.Label(root, text="Seleziona l'immagine:").grid(row=0, column=0, padx=10, pady=5)
image_path_entry = tk.Entry(root, width=50)
image_path_entry.grid(row=0, column=1, padx=10, pady=5)
tk.Button(root, text="Sfoglia", command=select_image_file).grid(row=0, column=2, padx=10, pady=5)

tk.Label(root, text="Seleziona la directory di output:").grid(row=1, column=0, padx=10, pady=5)
output_dir_entry = tk.Entry(root, width=50)
output_dir_entry.grid(row=1, column=1, padx=10, pady=5)
tk.Button(root, text="Sfoglia", command=select_output_directory).grid(row=1, column=2, padx=10, pady=5)

tk.Button(root, text="Avvia estrazione", command=start_extraction).grid(row=2, column=1, padx=10, pady=20)

root.mainloop()



<H1>SCRIPT PER L'ELABORAZONE DI SINGOLE IMMAGINI O IN BATCH </H1>

In [2]:
drawing = False
ix, iy = -1, -1
fx, fy = -1, -1

def process_image(image_path, output_path):
    if not os.path.exists(image_path):
        print(f"Errore: il file {image_path} non esiste.")
        return

    im = cv2.imread(image_path)

    if im is None:
        print(f"Errore durante la lettura dell'immagine: {image_path}")
        return

    base_name = os.path.splitext(os.path.basename(image_path))[0]

    # Converti in scala di grigi
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding
    bw = cv2.adaptiveThreshold(im_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 15, 8)

    # Denoise image
    bw = cv2.fastNlMeansDenoising(bw, None, 30, 7, 21)

    # Morphological operations to separate characters better
    kernel_dilate = np.ones((2, 2), np.uint8)
    kernel_erode = np.ones((1, 1), np.uint8)
    bw = cv2.erode(bw, kernel_erode, iterations=1)
    bw = cv2.dilate(bw, kernel_dilate, iterations=1)

    # Find contours to detect characters
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sorting contours from left to right
    contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

    # Save the image without rectangles
    cv2.imwrite(os.path.join(output_path, f'{base_name}_out_without_rectangles.png'), im_gray)

    # Draw rectangles on the original image
    im_with_rectangles = im.copy()
    for i, ctr in enumerate(contours):
        x, y, w, h = cv2.boundingRect(ctr)
        char_img = im_gray[y:y+h, x:x+w]
        cv2.imwrite(os.path.join(output_path, f'{base_name}_char_{i}.jpg'), char_img)
        cv2.rectangle(im_with_rectangles, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Save the images with and without rectangles
    cv2.imwrite(os.path.join(output_path, f'{base_name}_out.png'), im_with_rectangles)
    cv2.imwrite(os.path.join(output_path, f'{base_name}_out_bw.png'), bw)

def select_input_directory():
    dir_path = filedialog.askdirectory(title="Seleziona la directory di input")
    if dir_path:
        input_dir.set(dir_path)

def select_output_directory():
    dir_path = filedialog.askdirectory(title="Seleziona la directory di output")
    if dir_path:
        if mode_var.get() == 1:
            single_output_dir.set(dir_path)
        else:
            batch_output_dir.set(dir_path)

def process_all_images():
    input_path = input_dir.get()
    output_path = batch_output_dir.get()
    if not input_path or not output_path:
        messagebox.showwarning("Attenzione", "Per favore seleziona sia la directory di input che quella di output")
        return

    supported_formats = (".tif", ".jpg", ".jpeg", ".png")
    for filename in os.listdir(input_path):
        if filename.lower().endswith(supported_formats): 
            full_input_path = os.path.join(input_path, filename)
            full_output_path = os.path.join(output_path, filename.split('.')[0])
            os.makedirs(full_output_path, exist_ok=True)
            process_image(full_input_path, full_output_path) 

    messagebox.showinfo("Completato", "Elaborazione completata!")

def draw_rectangle(event, x, y, flags, param):
    global ix, iy, fx, fy, drawing, im_contours

    output_dir, characters, image_name = param

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            fx, fy = x, y
            im_temp = im_contours.copy()
            cv2.rectangle(im_temp, (ix, iy), (fx, fy), (0, 0, 255), 2)
            cv2.imshow('Contours with Bounding Boxes', im_temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        fx, fy = x, y

        if ix < 0:
            ix = 0
        if iy < 0:
            iy = 0
        if fx >= im.shape[1]:
            fx = im.shape[1] - 1
        if fy >= im.shape[0]:
            fy = im.shape[0] - 1
        
        if ix < fx and iy < fy:
            cv2.rectangle(im_contours, (ix, iy), (fx, fy), (0, 0, 255), 2)
            roi = im[iy:fy, ix:fx]
            output_path = os.path.join(output_dir, f'{image_name}_character_{len(characters) + 1}.png')
            cv2.imwrite(output_path, roi)
            characters.append(output_path)
            cv2.imshow('Contours with Bounding Boxes', im_contours)

def detect_characters(image_path, output_dir=None):
    global im, im_contours

    if output_dir is None:
        output_dir = 'extracted_characters'
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    im = cv2.imread(image_path)
    if im is None:
        print(f"Errore durante la lettura dell'immagine: {image_path}")
        return

    h, w = im.shape[:2]
    im_contours = im.copy()
    im = cv2.cvtColor(im_contours, cv2.COLOR_BGR2GRAY)
    window_name = 'Contours with Bounding Boxes'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, 800,1000)

    image_name = os.path.splitext(os.path.basename(image_path))[0]
    output_dir = single_output_dir.get()
    characters = []

    cv2.setMouseCallback(window_name, draw_rectangle, param=(output_dir, characters, image_name))

    while True:
        cv2.imshow(window_name, im_contours)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('0'):
            break

    cv2.destroyAllWindows()

def select_image_file():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.tif *.bmp")])
    if file_path:
        image_path_entry.delete(0, tk.END)
        image_path_entry.insert(0, file_path)

def start_extraction():
    image_path = image_path_entry.get()
    output_dir = single_output_dir.get()
    
    if not image_path or not output_dir:
        messagebox.showerror("Errore", "Seleziona un'immagine e una directory di output.")
        return
    
    detect_characters(image_path, output_dir)
    messagebox.showinfo("Successo", "Estrazione dei caratteri completata.")

def start_batch_processing():
    process_all_images()

# Configurazione dell'interfaccia grafica
root = tk.Tk()
root.title("Strumento di Elaborazione Immagini")
root.geometry("600x300")

def set_mode():
    mode = mode_var.get()
    if mode == 1:
        batch_frame.pack_forget()
        single_frame.pack(pady=10)
    else:
        single_frame.pack_forget()
        batch_frame.pack(pady=10)

mode_var = tk.IntVar(value=1)
tk.Radiobutton(root, text="Elaborazione Singola Immagine", variable=mode_var, value=1, command=set_mode).pack(anchor="w")
tk.Radiobutton(root, text="Elaborazione Batch", variable=mode_var, value=2, command=set_mode).pack(anchor="w")

single_frame = tk.Frame(root)
batch_frame = tk.Frame(root)

tk.Label(single_frame, text="Seleziona l'immagine:").grid(row=0, column=0, padx=10, pady=5)
image_path_entry = tk.Entry(single_frame, width=50)
image_path_entry.grid(row=0, column=1, padx=10, pady=5)
tk.Button(single_frame, text="Sfoglia", command=select_image_file).grid(row=0, column=2, padx=10, pady=5)

tk.Label(single_frame, text="Seleziona la directory di output:").grid(row=1, column=0, padx=10, pady=5)
single_output_dir = tk.StringVar()
output_dir_entry = tk.Entry(single_frame, textvariable=single_output_dir, width=50)
output_dir_entry.grid(row=1, column=1, padx=10, pady=5)
tk.Button(single_frame, text="Sfoglia", command=select_output_directory).grid(row=1, column=2, padx=10, pady=5)

tk.Button(single_frame, text="Avvia estrazione", command=start_extraction).grid(row=2, column=1, padx=10, pady=20)

tk.Label(batch_frame, text="Directory di input:").grid(row=0, column=0, padx=10, pady=5)
input_dir = tk.StringVar()
batch_output_dir = tk.StringVar()
input_entry = tk.Entry(batch_frame, textvariable=input_dir, width=50)
input_entry.grid(row=0, column=1, padx=10, pady=5)
input_button = tk.Button(batch_frame, text="Sfoglia", command=select_input_directory)
input_button.grid(row=0, column=2, padx=10, pady=5)

tk.Label(batch_frame, text="Directory di output:").grid(row=1, column=0, padx=10, pady=5)
output_entry = tk.Entry(batch_frame, textvariable=batch_output_dir, width=50)
output_entry.grid(row=1, column=1, padx=10, pady=5)
output_button = tk.Button(batch_frame, text="Sfoglia", command=select_output_directory)
output_button.grid(row=1, column=2, padx=10, pady=5)

tk.Button(batch_frame, text="Processa tutte le immagini", command=start_batch_processing).grid(row=2, column=1, padx=10, pady=20)

set_mode()
root.mainloop()



Errore durante la lettura dell'immagine: C:/Users/orazi/OneDrive/Desktop/tesi/cvl-database-1-1/testset/words/0363\0363-3-0-0-Mailüfterl.tif
Errore durante la lettura dell'immagine: C:/Users/orazi/OneDrive/Desktop/tesi/cvl-database-1-1/testset/words/0363\0363-3-10-1-Binär.tif
Errore durante la lettura dell'immagine: C:/Users/orazi/OneDrive/Desktop/tesi/cvl-database-1-1/testset/words/0363\0363-3-9-3-Mailüfterl.tif
Errore durante la lettura dell'immagine: C:/Users/orazi/OneDrive/Desktop/tesi/cvl-database-1-1/testset/words/0363\0363-6-1-5-schön.tif
Errore durante la lettura dell'immagine: C:/Users/orazi/OneDrive/Desktop/tesi/cvl-database-1-1/testset/words/0363\0363-6-7-4-für.tif


<H1>PER MIGLIORARE LA QUALITà DELL'IMMAGINE</H1>

In [3]:

contrast_factor = 2.0  
sharpness_factor = 2.0  

def process_image(image_path, output_path):
    try:
        im = Image.open(image_path)
    except Exception as e:
        print(f"Could not read image {image_path}: {e}")
        return
    

    enhancer = ImageEnhance.Contrast(im)
    image_enhanced = enhancer.enhance(contrast_factor)


    sharpener = ImageEnhance.Sharpness(image_enhanced)
    image_sharpened = sharpener.enhance(sharpness_factor)


    image_gray = image_sharpened.convert("L")
    

    image_gray.save(output_path)

def process_directory(input_dir, output_dir):

    for subdir, _, files in os.walk(input_dir):
        for file in files:
         
            file_path = os.path.join(subdir, file)
          
            relative_path = os.path.relpath(file_path, input_dir)
            output_path = os.path.join(output_dir, relative_path)
            
       
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
      
            process_image(file_path, output_path)


input_directory = r'C:\Users\orazi\OneDrive\Desktop\tesi\datiT\dati5-1' 
output_directory = r'C:\Users\orazi\OneDrive\Desktop\tesi\datiT\dati5-1' 

process_directory(input_directory, output_directory)



<H1>PER SALVARE CARTELLA CON IMMAGINI DENTRO</H1>

In [7]:
def rename_files_with_folder_name(input_folder, output_folder):

    for root, dirs, files in os.walk(input_folder):
        for file in files:

            file_path = os.path.join(root, file)
  
            folder_name = os.path.basename(root)

            new_file_name = f"{folder_name}_{file}"
            

            relative_path = os.path.relpath(root, input_folder)
            new_folder_path = os.path.join(output_folder, relative_path)
            
            if not os.path.exists(new_folder_path):
                os.makedirs(new_folder_path)
                

            new_file_path = os.path.join(new_folder_path, new_file_name)
            

            shutil.copy2(file_path, new_file_path)

input_folder = r"C:\Users\orazi\OneDrive\Desktop\LetterClassifier\toAdd\soggetto34"
output_folder = r"C:\Users\orazi\OneDrive\Desktop\LetterClassifier\toAdd\soggetto34A"

rename_files_with_folder_name(input_folder, output_folder)

<h1>PER SALVARE LE IMMAGINI IN UNA SINGOLA CARTELLA</h1>

In [8]:
def rename_files_with_folder_name(input_folder, output_folder):
    # Creare la cartella di output se non esiste
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    

    for root, dirs, files in os.walk(input_folder):
        for file in files:
         
            file_path = os.path.join(root, file)

            folder_name = os.path.basename(root)

            new_file_name = f"{folder_name}_{file}"
            
            new_file_path = os.path.join(output_folder, new_file_name)
         
            shutil.copy2(file_path, new_file_path)

input_folder = r"C:\Users\orazi\OneDrive\Desktop\tesi\soggetti2\0020"
output_folder = r"C:\Users\orazi\OneDrive\Desktop\tesi\soggetti\0024"

rename_files_with_folder_name(input_folder, output_folder)


In [7]:
import os
from PIL import Image

def resize_images_in_nested_folders(input_folder, output_folder=None, size=(50, 50)):
    if output_folder is None:
        output_folder = input_folder
    
    for root, dirs, files in os.walk(input_folder):
        # Costruire il percorso di output mantenendo la struttura delle cartelle
        relative_path = os.path.relpath(root, input_folder)
        output_dir = os.path.join(output_folder, relative_path)
        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                img_path = os.path.join(root, filename)
                try:
                    img = Image.open(img_path)
                    img = img.resize(size, Image.LANCZOS)
                    output_path = os.path.join(output_dir, filename)
                    img.save(output_path)
                    print(f'Resized and saved {filename} to {output_path}')
                except Exception as e:
                    print(f"Could not process file {filename}: {e}")

input_folder = r'dataset'
output_folder = 'dataset_resized'  # Optional, use None to overwrite input folder

resize_images_in_nested_folders(input_folder, output_folder)

Resized and saved a_0001-1-0-0-Imagine_character_3.png to dataset_resized\soggetto1\a\a_0001-1-0-0-Imagine_character_3.png
Resized and saved a_0001-1-0-1-a_character_1.png to dataset_resized\soggetto1\a\a_0001-1-0-1-a_character_1.png
Resized and saved a_0001-1-0-2-vast_character_2.png to dataset_resized\soggetto1\a\a_0001-1-0-2-vast_character_2.png
Resized and saved a_0001-1-0-5-paper_character_2.png to dataset_resized\soggetto1\a\a_0001-1-0-5-paper_character_2.png
Resized and saved a_0001-1-0-8-straight_character_4.png to dataset_resized\soggetto1\a\a_0001-1-0-8-straight_character_4.png
Resized and saved a_0001-1-1-1-Triangles_character_4.png to dataset_resized\soggetto1\a\a_0001-1-1-1-Triangles_character_4.png
Resized and saved a_0001-1-1-3-Squares_character_4.png to dataset_resized\soggetto1\a\a_0001-1-1-3-Squares_character_4.png
Resized and saved a_0001-1-1-4-Pentagons_character_6.png to dataset_resized\soggetto1\a\a_0001-1-1-4-Pentagons_character_6.png
Resized and saved a_0001-1-1

In [2]:
import os
import shutil

def copy_images(src_path, dest_path):
    # Creiamo la struttura delle cartelle di destinazione
    for letter in "abcdefghiklmnopqrstuvwxyz":
        letter_dir = os.path.join(dest_path, letter)
        os.makedirs(letter_dir, exist_ok=True)

    # Scorriamo le sottocartelle di soggetti
    for subject in os.listdir(src_path):
        subject_path = os.path.join(src_path, subject)
        if os.path.isdir(subject_path):
            # Scorriamo le sottocartelle delle lettere
            for letter in os.listdir(subject_path):
                letter_path = os.path.join(subject_path, letter)
                if os.path.isdir(letter_path) and letter in "abcdefghiklmnopqrstuvwxyz":
                    # Copiamo tutte le immagini nella cartella di destinazione
                    for file_name in os.listdir(letter_path):
                        file_path = os.path.join(letter_path, file_name)
                        if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                            dest_letter_path = os.path.join(dest_path, letter)
                            shutil.copy(file_path, dest_letter_path)

# Esempio di utilizzo
src_path = "dataset"
dest_path = "dataset3"
copy_images(src_path, dest_path)
